### Scikit-Learn의 문서 전처리 기능
- BOW(Bag of Words)
    - 문서를 숫자 벡터로 변환하는 가장 기본적인 방법이다.
    - 만약 단어가 문서 안에 있으면 1, 없으면 0으로 binarizing한다.

#### sklearn의 feature_extranction sub-package와 feature_extranction.text 서브 패키지는 다음과 같은 문서 전처리용 클래스를 제공한다.
- `DictVectorizer` : 각 단어의 수를 세어놓은 사전에서 BOW vector를 만든다.
- `CountVectorizer` : 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩한 벡터를 만든다.
- `TfidVectorizer` : CountVectorizer 와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BOW vectorㄹ르 만든다.
- `HashingVectorizer` : 해시 함수를 사용하여 적은 메모리와 빠른 속도로 BOW vector를 만든다

### DictVectorizer
- 문서에서 단어의 사용 빈도(fequency)를 나타내는 딕셔너리 정보를 입력받아 BOW 인코딩 수치 벡터로 변환한다.

In [9]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
D = [{'A': 1, 'B': 2}, {'B': 3, 'C': 1}]# 각 단어의 수를 세어놓은 사전
X = v.fit_transform(D)#에서 BOW vector를 만든다.
X

array([[1., 2., 0.],
       [0., 3., 1.]])

In [10]:
v.feature_names_

['A', 'B', 'C']

In [12]:
v.transform({'C': 4, 'D': 3})

array([[0., 0., 4.]])

### CountVectorizer
- 문서를 토큰 리스트로 변환한다.
- 각 문서에서 토큰의 출현 빈도를 센다.
- 각 문서를 BOW 인코딩 벡터로 변환한다.

DictVectorizer는 각 단어의 빈도수를 담은 딕셔너리를 인자로 받아서 BOW vector로 변환하지만, CountVectorizer는 문서(corpus)를 받아서 직접 빈도를 세서 이를  BOW vector로 변환시킨다.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',    
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [14]:
vect.transform(['This is the second document.']).toarray()

array([[0, 1, 0, 1, 0, 0, 1, 1, 0, 1]])

In [15]:
vect.transform(['Something completely new.']).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
vect.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]])

위에서 볼 수 있다시피 fit() -> vocabulary_ 메소드를 사용하면 frequecy dictionary return 을 하고, transform.toarray() 메소드를 사용하면 BOW vector transformation을 한다.

- CountVectorizer는 이러한 작업을 하기 위한 다음과 같은 인수를 가질 수 있다.

- stop_words : 문자열 {‘english’}, 리스트 또는 None (디폴트)
- stop words 목록.‘english’이면 영어용 스탑 워드 사용.
- analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수
    - 단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램
- token_pattern : string
    - 토큰 정의용 정규 표현식
- tokenizer : 함수 또는 None (디폴트)
    - 토큰 생성 함수 .
- ngram_range : (min_n, max_n) 튜플
    - n-그램 범위
- max_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
    - 단어장에 포함되기 위한 최대 빈도
- min_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
    - 단어장에 포함되기 위한 최소 빈도

### Stop Words
- 문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 보통 영어의 관사나 접속사 한국어의 조사 등이 여기에 해당한다.

In [18]:
vect = CountVectorizer(stop_words=['and','is','the','this']).fit(corpus)
vect.vocabulary_

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [35]:
vect = CountVectorizer(stop_words="english").fit(corpus)
vect.vocabulary_

{'document': 0, 'second': 1}

In [36]:
vect.transform(corpus).toarray()

array([[1, 0],
       [1, 2],
       [0, 0],
       [1, 0],
       [1, 0]])

### 토큰
- analyzer, tokenizer, token_pattern 등의 인수로 사용할 토큰 생성기를 선택할 수 있다.

나는 character으로 받겠다라면 아래를 따른다.(with token_pattern)

In [20]:
vect = CountVectorizer(analyzer='char').fit(corpus)
vect.vocabulary_

{'t': 16,
 'h': 8,
 'i': 9,
 's': 15,
 ' ': 0,
 'e': 6,
 'f': 7,
 'r': 14,
 'd': 5,
 'o': 13,
 'c': 4,
 'u': 17,
 'm': 11,
 'n': 12,
 '.': 1,
 'a': 3,
 '?': 2,
 'l': 10}

나는 t로 시작하는 word를 받겠다하면 아래와 같이 regular expression 을 사용할 수 있다.(with token_pattern)

In [21]:
vect = CountVectorizer(token_pattern="t\w+").fit(corpus)
vect.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

nltk에서 제공하는 메소드 word_tokenize 함수를 쓰겠다라면 아래를 따른다. (with tokenzier)

In [22]:
import nltk

vect = CountVectorizer(tokenizer=nltk.word_tokenize).fit(corpus)
vect.vocabulary_

{'this': 11,
 'is': 5,
 'the': 9,
 'first': 4,
 'document': 3,
 '.': 0,
 'second': 8,
 'and': 2,
 'third': 10,
 'one': 7,
 '?': 1,
 'last': 6}

#### n-그램은 단어장 생성에 사용할 토큰의 크기를 결정한다. 모노그램(1-그램)은 토큰 하나만 단어로 사용하며 바이그램(2-그램)은 두 개의 연결된 토큰을 하나의 단어로 사용한다.

In [23]:
vect = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
vect.vocabulary_

{'this is': 12,
 'is the': 2,
 'the first': 7,
 'first document': 1,
 'the second': 9,
 'second second': 6,
 'second document': 5,
 'and the': 0,
 'the third': 10,
 'third one': 11,
 'is this': 3,
 'this the': 13,
 'the last': 8,
 'last document': 4}

In [24]:
vect = CountVectorizer(ngram_range=(1, 2), token_pattern="t\w+").fit(corpus)
vect.vocabulary_
#1~2

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

#### 빈도수
- max_df , min_df 인수를 사용하여 문서에서 토큰이 나타난 횟수를 기준으로 단어장을 구성할 수도 있다.

In [26]:
vect = CountVectorizer(max_df = 4, min_df = 2).fit(corpus)
vect.vocabulary_ , vect.stop_words_

({'this': 3, 'is': 2, 'first': 1, 'document': 0},
 {'and', 'last', 'one', 'second', 'the', 'third'})

In [27]:
vect.transform(corpus).toarray().sum(axis=0)

array([4, 2, 3, 3])

### TF - IDF ( Term Frequency - Inverse Document Frequency)
- 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법이다.
- tf(d,t) : term-frequency (특정한 단어의 빈도수)
- idf(t) : inverse document frequency (특정한 단어가 들어 있는 문서의 수에 반비례하는 수)
- `tf-idf(d,f) = tf(d,t);idf(t)`
- `idf(d,t) = log(n/(1+df(t))`

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidv = TfidfVectorizer().fit(corpus)
tfidv.transform(corpus).toarray()

array([[0.        , 0.38947624, 0.55775063, 0.4629834 , 0.        ,
        0.        , 0.        , 0.32941651, 0.        , 0.4629834 ],
       [0.        , 0.24151532, 0.        , 0.28709733, 0.        ,
        0.        , 0.85737594, 0.20427211, 0.        , 0.28709733],
       [0.55666851, 0.        , 0.        , 0.        , 0.        ,
        0.55666851, 0.        , 0.26525553, 0.55666851, 0.        ],
       [0.        , 0.38947624, 0.55775063, 0.4629834 , 0.        ,
        0.        , 0.        , 0.32941651, 0.        , 0.4629834 ],
       [0.        , 0.45333103, 0.        , 0.        , 0.80465933,
        0.        , 0.        , 0.38342448, 0.        , 0.        ]])

### Hasing Trick
- CountVectorizer 는 모든 작업을 메모리 상에서 수행하므로 처리할 문서의 크기가 커지면 속도가 느려지거나 실행이 안된다.
- 이때 HasingVectorizer를 사용하면 해시 함수를 사용하여 단어에 대한 인덱스 번호를 생성하기 때문에 메모리 및 실행 시간을 줄일 수 있다.

In [30]:
from sklearn.datasets import fetch_20newsgroups
twenty =fetch_20newsgroups()
len(twenty.data)

11314

In [31]:
%time CountVectorizer().fit(twenty.data).transform(twenty.data);

CPU times: user 4.39 s, sys: 34.8 ms, total: 4.42 s
Wall time: 4.42 s


<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=300000)
# n_features : Convert a collection of text documents to a matrix of token occurrences

In [34]:
%time hv.transform(twenty.data);

CPU times: user 4 s, sys: 98.9 ms, total: 4.1 s
Wall time: 1.9 s


<11314x300000 sparse matrix of type '<class 'numpy.float64'>'
	with 1786336 stored elements in Compressed Sparse Row format>